<a href="https://colab.research.google.com/github/kyleplhm/cloud_computing/blob/main/cloud_week_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>